In [1]:
# To check if gpu is available or not 

import torch
print(torch.cuda.is_available())

True


In [2]:
# Importing all the required library

import os
from ultralytics import YOLO
from keras_facenet import FaceNet
from sklearn.metrics.pairwise import cosine_similarity
import cv2
import numpy as np

In [3]:
# Load models with GPU support if available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

Using device: cuda


In [4]:
# Loading yolo v8 model for face detection

face_detector = YOLO('yolov8m-face.pt').to(device)

In [5]:
# Loading facenet model 

embedder = FaceNet()

In [6]:
# Image preprocessing

def preprocess_img(img):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    equalized = cv2.equalizeHist(gray)
    return cv2.cvtColor(equalized, cv2.COLOR_GRAY2RGB)

In [7]:
# Finding embeddings from image 

def get_embeddings_from_folder(folder_path):
    embeddings = []
    for filename in os.listdir(folder_path):
        if filename.endswith(('.jpg', '.jpeg', '.png')):
            img_path = os.path.join(folder_path, filename)
            img = cv2.imread(img_path)
            if img is not None:
                img = preprocess_img(img)
                embedding = embedder.embeddings([img])
                embeddings.append((filename, embedding[0]))
    return embeddings

In [8]:
# face recognition

def recognize_face(embeddings, known_embeddings, threshold=0.5):
    for filename, known_embedding in known_embeddings:
        sim = cosine_similarity([embedding], [known_embedding])[0][0]
        if sim > threshold:
            return filename
    return "Unknown"

In [9]:
# folder containing images of particular person

person_folder = "C:/Users/khadi/Desktop/RealTime_face recognition/img_dataset/Rachet"

In [11]:
# Get embeddings for all images in the folder

person_embeddings = get_embeddings_from_folder(person_folder)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 208ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 209ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 194ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 237ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 210ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 248ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 213ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 163ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 198ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 

In [ ]:
# Initialize video capture

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Detect faces
    results = face_detector(frame)

    for result in results:
        boxes = result.boxes
        for box in boxes:  # Access detected boxes (Multiple faces support)
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # Get the coordinates of the bounding box
            conf = box.conf[0]  # Confidence score
            face = frame[y1:y2, x1:x2]

            # Preprocess face before embedding
            face = preprocess_img(face)

            # Get face embedding using FaceNet
            if face.size != 0:  # Make sure face isn't empty
                embedding = embedder.embeddings([face])[0]  # Get the first embedding

                # Recognize face
                recognized_name = recognize_face(embedding, person_embeddings)

                # Draw bounding box and label
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, recognized_name, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2)

    cv2.imshow("Face Recognition", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


0: 736x960 2 faces, 189.3ms
Speed: 250.9ms preprocess, 189.3ms inference, 604.4ms postprocess per image at shape (1, 3, 736, 960)
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step

0: 736x960 2 faces, 271.5ms
Speed: 14.0ms preprocess, 271.5ms inference, 8.2ms postprocess per image at shape (1, 3, 736, 960)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step

0: 736x960 2 faces, 82.6ms
Speed: 11.0ms preprocess, 82.6ms inference, 7.0ms postprocess per image at shape (1, 3, 736, 960)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 159ms/step

0: 736x960 2 faces, 83.5ms
Speed: 11.7ms preprocess, 83.5ms inference, 4.0ms postprocess per image at shape (1, 3, 736, 960)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step

0: 736x960 2 faces, 84.7ms
Speed: 10.4ms preprocess, 84.7ms inference, 4.0ms postprocess per image at shape (1, 3, 736, 960)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
1/1 ━━━━━━━━━━━

In [13]:
# Testing part

import os
import cv2
from sklearn.metrics import accuracy_score

# Function to run model on test dataset
def test_accuracy_on_dataset(test_folder_path, known_embeddings, threshold=0.5):
    y_true = []  # Ground truth labels
    y_pred = []  # Predicted labels
    
    for filename in os.listdir(test_folder_path):
        if filename.endswith(('.jpg', '.jpeg', '.png')):
            img_path = os.path.join(test_folder_path, filename)
            img = cv2.imread(img_path)
            if img is not None:
                # Assume the filename (without extension) is the actual label
                actual_label = filename.split('.')[0]
                y_true.append(actual_label)

                # Get face embedding
                embedding = embedder.embeddings([img])[0]
                
                # Recognize face
                predicted_name = recognize_face(embedding, known_embeddings, threshold=threshold)
                y_pred.append(predicted_name)

    # Calculate accuracy
    accuracy = accuracy_score(y_true, y_pred)
    print(f"Accuracy: {accuracy * 100:.2f}%")

# Usage
test_folder = "C:/Users/khadi/Desktop/RealTime_face recognition/img_dataset/Rachet"  # Replace with your test dataset path
test_accuracy_on_dataset(test_folder, person_embeddings)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 166ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
for filename in os.listdir(test_folder):
    if filename.endswith(('.jpg', '.jpeg', '.png')):
        # Existing code...
        predicted_name = recognize_face(embedding, known_embeddings, threshold=threshold)
        print(f"Filename: {actual_label}, Predicted: {predicted_name}")  # Debugging line
        y_pred.append(predicted_name)